In [1]:

import numpy as np
import pandas as pd
from scipy.stats import rankdata

matrix = np.array([
    [0.168, 0.083, 0.107, 0.111],
    [0.090, 0.078, 0.103, 0.102],
    [0.060, 0.075, 0.083, 0.088],
    [0.073, 0.091, 0.242, 0.078],
    [0.249, 0.063, 0.120, 0.068],
    [0.033, 0.139, 0.020, 0.144],
    [0.025, 0.095, 0.025, 0.148],
    [0.032, 0.130, 0.116, 0.105],
    [0.091, 0.103, 0.128, 0.089], 
    [0.012, 0.144, 0.044, 0.069]
])

sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

def marcoss(matrix, weights, criteria_types, sites):
    # Step 1: Define AI and AAI
    ai = np.where(np.array(criteria_types) == 'max', 
                  np.max(matrix, axis=0), 
                  np.min(matrix, axis=0))
    
    aai = np.where(np.array(criteria_types) == 'max', 
                   np.min(matrix, axis=0), 
                   np.max(matrix, axis=0))
    
    # Step 2: Extend matrix with AI and AAI
    extended = np.vstack([matrix, ai, aai])

    # Step 3: Normalize extended matrix
    norm_extended = np.zeros_like(extended, dtype=float)
    for j in range(extended.shape[1]):
        if criteria_types[j] == 'max':
            norm_extended[:, j] = extended[:, j] / ai[j]
        else:
            norm_extended[:, j] = aai[j] / extended[:, j]
    
    # Step 4: Apply weights to normalized matrix
    weighted_extended = norm_extended * weights

    # Step 5: Compute S_i, S_AI, S_AAI
    S = weighted_extended.sum(axis=1)
    S_ai, S_aai = S[-2], S[-1]  # Last two rows are AI and AAI
    S_alts = S[:-2]             # Alternatives only

    # Step 6: Compute utility degrees
    K_plus = S_alts / S_ai
    K_minus = S_alts / S_aai

    # Step 7: Final utility function as compromise score
    utility = (K_plus + K_minus) / (
        1 + (1 - K_plus) / K_plus + (1 - K_minus) / K_minus
    )

    # Step 8: Rank alternatives
    ranks = rankdata(-utility, method='min')

    # Step 9: Return results
    return pd.DataFrame({
        'Site': sites,
        'Utility': utility,
        'Rank': ranks
    }).sort_values(by='Rank').reset_index(drop=True)

# Run MARCOS and show result
result_df = marcoss(matrix, weights, criteria_types, sites)
print(result_df)

     Site   Utility  Rank
0   Site4  5.632025     1
1   Site5  3.672769     2
2   Site9  2.846530     3
3   Site1  2.788960     4
4   Site8  2.652902     5
5   Site2  1.632621     6
6  Site10  1.495036     7
7   Site6  1.259349     8
8   Site3  1.061169     9
9   Site7  0.602326    10
